# Texas Tow Trucks

We're going to scrape some [tow trucks in Texas](https://www.tdlr.texas.gov/tools_search/).

## Import your imports

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

## Search for the TLDR Number `006179570C`, and scrape the information on that company

Using [license information system](https://www.tdlr.texas.gov/tools_search/), find information about the tow truck number above, displaying the

- The business name
- Owner/operator
- Phone number
- License status (Active, Expired, Etc)
- Physical address

If you can't figure a 'nice' way to locate something, your two last options might be:

- **Find a "parent" element, then dig inside**
- **Find all of a type of element** (like we did with `td` before) and get the `[0]`, `[1]`, `[2]`, etc
- **XPath** (inspect an element, Copy > Copy XPath)

These kinds of techniques tend to break when you're on other result pages, but... maybe not! You won't know until you try.

> - *TIP: When you use xpath, you CANNOT use double quotes or Python will get confused. Use single quotes.*
> - *TIP: You can clean your data up if you want to, or leave it dirty to clean later*
> - *TIP: The address part can be tough, but you have a few options. You can use a combination of `.split` and list slicing to clean it now, or clean it later in the dataframe with regular expressions. Or other options, too, probably*

In [2]:
driver = webdriver.Chrome()
driver.get("https://www.tdlr.texas.gov/tools_search/")

In [3]:
text_input = driver.find_element_by_id('mcrdata')
text_input.send_keys('006179570C')

In [4]:
search_button = driver.find_element_by_id('submit3')
search_button.click()

In [5]:
#The business name
#Owner/operator
#Phone number
#License status (Active, Expired, Etc)
#Physical address

name = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[2]/td[1]')
name.text

owner = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[3]/td[1]')
owner.text

phone = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[4]/td[1]')
phone.text

license = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[3]/tbody/tr[1]/td[2]')
license.text

address = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[3]/tbody/tr[2]/td[2]')
address.text.split('\n')[-3:]

['Physical:', '13619 BRETT JACKSON RD.', 'FORT WORTH, TX. 76179']

# Adapt this to work inside of a single cell

Double-check that it works. You want it to print out all of the details.

In [6]:
datapoints = []

results = driver.find_elements_by_xpath('//*[@id="t1"]/tbody/tr/td')
for result in results:
#     print("---------")
    datapoint = {}
    
    name = result.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[2]/td[1]')
    datapoint['name'] = name.text
    
    owner = result.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[3]/td[1]')
    datapoint['owner'] = owner.text
    
    phone = result.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[4]/td[1]')
    datapoint['phone'] = phone.text

    license = result.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[3]/tbody/tr[1]/td[2]')
    datapoint['license'] = license.text

    address = result.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[3]/tbody/tr[2]/td[2]')
    datapoint['address'] = address.text.split('\n')[-3:]
    
    datapoints.append(datapoint)
    
datapoints

[{'name': 'Name:   B.D. SMITH TOWING',
  'owner': 'Owner/Officer:   BRANDT SMITH / OWNER',
  'phone': 'Phone:   8173330706',
  'license': 'Status:  Expired',
  'address': ['Physical:',
   '13619 BRETT JACKSON RD.',
   'FORT WORTH, TX. 76179']}]

# Using .apply to find data about SEVERAL tow truck companies

The file `trucks-subset.csv` has information about the trucks, we'll use it to find the pages to scrape.

### Open up `trucks-subset.csv` and save it into a dataframe

In [7]:
import pandas as pd

In [8]:
df = pd.read_csv("trucks-subset.csv")

In [9]:
df

,TDLR Number
0,006507931C
1,006179570C
2,006502097C


## Go through each row of the dataset, displaying the URL you will need to scrape for the information on that row

You don't have to actually use the search form for each of these - look at the URL you're on, it has the number in it!

For example, one URL might look like `https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006495492C`.

- *TIP: Use .apply and a function*
- *TIP: Unlike the Yelp example, you'll need to build this URL from pieces*
- *TIP: You probably don't want to `print` unless you're going to fix it for the next question 
- *TIP: pandas won't showing you the entire url! Run `pd.set_option('display.max_colwidth', -1)` to display aaaalll of the text in a cell*

In [10]:
def get_truck_info(row):
    url = ('https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=' + row['TDLR Number'])

    return url

### Save this URL into a new column of your dataframe, called `url`

- *TIP: Use a function and `.apply`*
- *TIP: Be sure to use `return`*

In [11]:
pd.set_option('display.max_colwidth', -1)
df['url'] = df.apply(get_truck_info, axis=1)
df.head()

,TDLR Number,url
0,006507931C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C
1,006179570C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C
2,006502097C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006502097C


## Go through each row of the dataset, printing out information about each tow truck company.

Now will be **scraping** inside of your function.

- The business name
- Owner/operator
- Phone number
- License status (Active, Expired, Etc)
- Physical address

Just print it out for now.

- *TIP: use .apply*
- *TIP: You'll be using the code you wrote before, but converted into a function*
- *TIP: Remember how the TDLR Number is in the URL? You don't need to do the form submission if you don't want!*
- *TIP: Make sure you adjust any variables so you don't scrape the same page again and again*

In [18]:
def get_truck_info(row):
    driver.get(row['url'])

    
    name = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[2]/td[1]').text
    owner = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[3]/td[1]').text
    phone = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[4]/td[1]').text
    license = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[3]/tbody/tr[1]/td[2]').text
    address = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[3]/tbody/tr[2]/td[2]').text.split('\n')[-3:]


    return name, owner, phone, license, address

In [19]:
complete_df = df.apply(get_truck_info, axis=1)
complete_df.head()

0    (Name:   AUGUSTUS E SMITH, Owner/Officer:   AUGUSTUS EUGENE SMITH / OWNER, Phone:   9032276464, Status:  Active, [Physical:, 103 N MAIN ST, BONHAM, TX. 75418])       
1    (Name:   B.D. SMITH TOWING, Owner/Officer:   BRANDT SMITH / OWNER, Phone:   8173330706, Status:  Expired, [Physical:, 13619 BRETT JACKSON RD., FORT WORTH, TX. 76179])
2    (Name:   BARRY MICHAEL SMITH, Owner/Officer:   BARRY MICHAEL SMITH / OWNER, Phone:   8066544404, Status:  Active, [Physical:, 4501 W CEMETERY RD, CANYON, TX. 79015]) 
dtype: object

## Scrape the following information for each row of the dataset, and save it into new columns in your dataframe.

- The business name
- Owner/operator
- Phone number
- License status (Active, Expired, Etc)
- Physical address

It's basically what we did before, but using the function a little differently.

- *TIP: Same as above, but you'll be returning a `pd.Series` and the `.apply` line is going to be a lot longer*
- *TIP: Save it to a new dataframe!*
- *TIP: Make sure you change your `df` variable names correctly if you're cutting and pasting - there are a few so it can get tricky*

In [20]:
def get_truck_info(row):
    driver.get(row['url'])

    
    name = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[2]/td[1]').text
    owner = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[3]/td[1]').text
    phone = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[4]/td[1]').text
    license = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[3]/tbody/tr[1]/td[2]').text
    address = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[3]/tbody/tr[2]/td[2]').text.split('\n')[-3:]


    return pd.Series({
        'name': name,
        'owner': owner,
        'phone': phone,
        'license': license,
        'address': address
    })

In [22]:
complete_df = df.apply(get_truck_info, axis=1).join(df)
complete_df.head()

,name,owner,phone,license,address,TDLR Number,url
0,Name: AUGUSTUS E SMITH,Owner/Officer: AUGUSTUS EUGENE SMITH / OWNER,Phone: 9032276464,Status: Active,"[Physical:, 103 N MAIN ST, BONHAM, TX. 75418]",006507931C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C
1,Name: B.D. SMITH TOWING,Owner/Officer: BRANDT SMITH / OWNER,Phone: 8173330706,Status: Expired,"[Physical:, 13619 BRETT JACKSON RD., FORT WORTH, TX. 76179]",006179570C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C
2,Name: BARRY MICHAEL SMITH,Owner/Officer: BARRY MICHAEL SMITH / OWNER,Phone: 8066544404,Status: Active,"[Physical:, 4501 W CEMETERY RD, CANYON, TX. 79015]",006502097C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006502097C


### Save your dataframe as a CSV named `tow-trucks-extended.csv`

In [25]:
complete_df.to_csv("tow-trucks-extended.csv", index=False)

### Re-open your dataframe to confirm you didn't save any extra weird columns

In [26]:
dataframe = pd.read_csv("tow-trucks-extended.csv")
dataframe

,name,owner,phone,license,address,TDLR Number,url
0,Name: AUGUSTUS E SMITH,Owner/Officer: AUGUSTUS EUGENE SMITH / OWNER,Phone: 9032276464,Status: Active,"['Physical:', '103 N MAIN ST', 'BONHAM, TX. 75418']",006507931C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C
1,Name: B.D. SMITH TOWING,Owner/Officer: BRANDT SMITH / OWNER,Phone: 8173330706,Status: Expired,"['Physical:', '13619 BRETT JACKSON RD.', 'FORT WORTH, TX. 76179']",006179570C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C
2,Name: BARRY MICHAEL SMITH,Owner/Officer: BARRY MICHAEL SMITH / OWNER,Phone: 8066544404,Status: Active,"['Physical:', '4501 W CEMETERY RD', 'CANYON, TX. 79015']",006502097C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006502097C


## Process the entire `tow-trucks.csv` file

We just did it on a short subset so far. Now try it on all of the tow trucks. **Save as the same filename as before**

In [27]:
df = pd.read_csv("tow-trucks.csv")

In [28]:
df

,TDLR Number
0,006507931C
1,006179570C
2,006502097C
3,006494912C
4,0649468VSF
5,006448786C
6,0648444VSF
7,0651667VSF
8,006017767C
9,006495492C


In [30]:
def get_truck_info(row):
    url = ('https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=' + row['TDLR Number'])

    return url

In [32]:
pd.set_option('display.max_colwidth', -1)
df['url'] = df.apply(get_truck_info, axis=1)
df

,TDLR Number,url
0,006507931C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C
1,006179570C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C
2,006502097C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006502097C
3,006494912C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006494912C
4,0649468VSF,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=0649468VSF
5,006448786C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006448786C
6,0648444VSF,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=0648444VSF
7,0651667VSF,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=0651667VSF
8,006017767C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006017767C
9,006495492C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006495492C


In [33]:
def get_truck_info(row):
    driver.get(row['url'])

    
    name = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[2]/td[1]').text
    owner = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[3]/td[1]').text
    phone = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[2]/tbody/tr[4]/td[1]').text
    license = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[3]/tbody/tr[1]/td[2]').text
    address = driver.find_element_by_xpath('//*[@id="t1"]/tbody/tr/td/font/table[3]/tbody/tr[2]/td[2]').text.split('\n')[-3:]


    return pd.Series({
        'name': name,
        'owner': owner,
        'phone': phone,
        'license': license,
        'address': address
    })

In [34]:
complete_df = df.apply(get_truck_info, axis=1).join(df)
complete_df

,name,owner,phone,license,address,TDLR Number,url
0,Name: AUGUSTUS E SMITH,Owner/Officer: AUGUSTUS EUGENE SMITH / OWNER,Phone: 9032276464,Status: Active,"[Physical:, 103 N MAIN ST, BONHAM, TX. 75418]",006507931C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C
1,Name: B.D. SMITH TOWING,Owner/Officer: BRANDT SMITH / OWNER,Phone: 8173330706,Status: Expired,"[Physical:, 13619 BRETT JACKSON RD., FORT WORTH, TX. 76179]",006179570C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C
2,Name: BARRY MICHAEL SMITH,Owner/Officer: BARRY MICHAEL SMITH / OWNER,Phone: 8066544404,Status: Active,"[Physical:, 4501 W CEMETERY RD, CANYON, TX. 79015]",006502097C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006502097C
3,Name: HEATH SMITH,Owner/Officer: HEATH A SMITH / OWNER,Phone: 940-552-0687,Status: Expired,"[Physical:, 1529 WILBARGER ST, VERNON, TX. 76384]",006494912C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006494912C
4,Name: HEATH SMITH,Owner/Officer: HEATH A SMITH / OWNER,Phone: 9405520687,Status: Expired,"[Physical:, 1529 WILBARGER ST, VERNON, TX. 76384]",0649468VSF,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=0649468VSF
5,Name: HYSMITH AUTOMOTIVE,Owner/Officer: WILLIAM THOMAS HYSMITH / PRESIDENT,Owner/Officer: ASHLEY ERIN HYSMITH / TREASURER,Status: Active,"[Physical:, 1210 US 380 BYPASS, GRAHAM, TX. 76450]",006448786C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006448786C
6,Name: HYSMITH AUTOMOTIVE & TRUCK REPAIR INC,Owner/Officer: HYSMITH ERIN ASHLEY / TREASURER,Owner/Officer: WILLIAM THOMAS HYSMITH / PRESIDENT,Status: Suspended,"[Physical:, 927 LOVING HWY, GRAHAM, TX. 76450]",0648444VSF,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=0648444VSF
7,Name: HYSMITH AUTOMOTIVE & TRUCK REPAIR INC,Owner/Officer: WILLIAM THOMAS HYSMITH / PRESIDENT,Owner/Officer: ASHLEY ERIN HYSMITH / TREASURER,Status: Active,"[Physical:, 1210 380 BYPASS, GRAHAM, TX. 76450]",0651667VSF,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=0651667VSF
8,Name: JEFF & WENDY SMITH,Owner/Officer: JEFF SMITH / PARTNER,Owner/Officer: WENDY SMITH / PARTNER,Status: Suspended,"[Physical:, 10842 FM 2138 N, JACKSONVILLE, TX. 75766]",006017767C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006017767C
9,Name: JEFF SMITH,Owner/Officer: JEFFREY JOHN SMITH / OWNER,Phone: 8324354670,Status: Active,"[Physical:, 4338 HARVEY RD, CROSBY, TX. 77532]",006495492C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006495492C


In [35]:
complete_df.to_csv("tow-trucks.csv", index=False)

In [36]:
dataframe = pd.read_csv("tow-trucks.csv")
dataframe

,name,owner,phone,license,address,TDLR Number,url
0,Name: AUGUSTUS E SMITH,Owner/Officer: AUGUSTUS EUGENE SMITH / OWNER,Phone: 9032276464,Status: Active,"['Physical:', '103 N MAIN ST', 'BONHAM, TX. 75418']",006507931C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C
1,Name: B.D. SMITH TOWING,Owner/Officer: BRANDT SMITH / OWNER,Phone: 8173330706,Status: Expired,"['Physical:', '13619 BRETT JACKSON RD.', 'FORT WORTH, TX. 76179']",006179570C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C
2,Name: BARRY MICHAEL SMITH,Owner/Officer: BARRY MICHAEL SMITH / OWNER,Phone: 8066544404,Status: Active,"['Physical:', '4501 W CEMETERY RD', 'CANYON, TX. 79015']",006502097C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006502097C
3,Name: HEATH SMITH,Owner/Officer: HEATH A SMITH / OWNER,Phone: 940-552-0687,Status: Expired,"['Physical:', '1529 WILBARGER ST', 'VERNON, TX. 76384']",006494912C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006494912C
4,Name: HEATH SMITH,Owner/Officer: HEATH A SMITH / OWNER,Phone: 9405520687,Status: Expired,"['Physical:', '1529 WILBARGER ST', 'VERNON, TX. 76384']",0649468VSF,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=0649468VSF
5,Name: HYSMITH AUTOMOTIVE,Owner/Officer: WILLIAM THOMAS HYSMITH / PRESIDENT,Owner/Officer: ASHLEY ERIN HYSMITH / TREASURER,Status: Active,"['Physical:', '1210 US 380 BYPASS', 'GRAHAM, TX. 76450']",006448786C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006448786C
6,Name: HYSMITH AUTOMOTIVE & TRUCK REPAIR INC,Owner/Officer: HYSMITH ERIN ASHLEY / TREASURER,Owner/Officer: WILLIAM THOMAS HYSMITH / PRESIDENT,Status: Suspended,"['Physical:', '927 LOVING HWY', 'GRAHAM, TX. 76450']",0648444VSF,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=0648444VSF
7,Name: HYSMITH AUTOMOTIVE & TRUCK REPAIR INC,Owner/Officer: WILLIAM THOMAS HYSMITH / PRESIDENT,Owner/Officer: ASHLEY ERIN HYSMITH / TREASURER,Status: Active,"['Physical:', '1210 380 BYPASS', 'GRAHAM, TX. 76450']",0651667VSF,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=0651667VSF
8,Name: JEFF & WENDY SMITH,Owner/Officer: JEFF SMITH / PARTNER,Owner/Officer: WENDY SMITH / PARTNER,Status: Suspended,"['Physical:', '10842 FM 2138 N', 'JACKSONVILLE, TX. 75766']",006017767C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006017767C
9,Name: JEFF SMITH,Owner/Officer: JEFFREY JOHN SMITH / OWNER,Phone: 8324354670,Status: Active,"['Physical:', '4338 HARVEY RD', 'CROSBY, TX. 77532']",006495492C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006495492C
